In [2]:
Dict = {"Price": [],
        "sqft": [],
        "Bedrooms": []
        }

In [3]:
Dict["sqft"].append("12ft")

In [4]:
Dict

{'Price': [], 'sqft': ['12ft'], 'Bedrooms': []}

In [5]:
List_Price = [12000,14522,156728,167289]
Dict["Price"].append(List_Price)

In [6]:
Dict

{'Price': [[12000, 14522, 156728, 167289]], 'sqft': ['12ft'], 'Bedrooms': []}

In [80]:
import selenium
from selenium import webdriver
import pandas as pd
from selenium.webdriver import Chrome
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import time


class Scraper:
    def __init__(self, url: str = 'https://www.propertypal.com') -> None:
        self.driver = Chrome(ChromeDriverManager().install())
        self.driver.get(url)
        
    def ivan_accept_cookies(self, xpath:str = '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]'):
        
        try:
            time.sleep(1)
            WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))

            button = self.driver.find_element(By.XPATH, xpath)
            button.click()
        except TimeoutException:
            print("No Cookies Found")

    def accept_cookies(self, xpath:str = '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]'):
        button = self.driver.find_element(By.XPATH, xpath)
        button.click()

    
    def search_word(self, xpath, text):

        search = self.driver.find_element(By.XPATH, xpath)
        self.driver.implicitly_wait(10)
        ActionChains(self.driver).move_to_element(search).click(search).perform()
        ActionChains(self.driver).send_keys(text).perform()

    def scroll_down(self):
        
        self.driver.execute_script("window.scrollTo(0, 540)") 
    
    def search_rent(self):
        rent = self.driver.find_element_by_xpath('//*[@id="searchForm"]/div/div[2]/button[2]')
        rent.click()

    def search_sale(self):
        rent = self.driver.find_element_by_xpath('//*[@id="searchForm"]/div/div[2]/button[1]')
        rent.click()

    def button_click(self,xpath):
        button = self.driver.find_element_by_xpath(xpath)
        button.click()
    
    def find_container(self, xpath: str = '//*[@id="body"]/div[3]/div/div[1]/div/ul'):
        self.container = self.driver.find_element(By.XPATH, xpath)
        return self.container

##
# NAVIGATE THE WEBSITE
##

if __name__ == "__main__":
    bot = Scraper()
    bot.ivan_accept_cookies()
    time.sleep(2)
    bot.button_click('//a[@href="/login"]')
    time.sleep(2)
    bot.search_word('//input[@placeholder="Email address"]','sopranotony233@gmail.com')
    time.sleep(2)
    bot.search_word('//input[@placeholder="Password"]','sopranotony321')
    time.sleep(2)
    bot.button_click('//*[@id="root"]/div/div/div/div/div/div[1]/div/div[2]/div[2]/form/button')
    time.sleep(2)
    bot.button_click('//a[@class="mainnav-logo"]')  
    time.sleep(2)
    bot.search_word('//*[@id="searchForm"]/div/div[1]', 'bt4')
    time.sleep(2)
    bot.search_rent()
    time.sleep(2)
    

    #CREATE THE LIST OF LINKS
    list_links = []
    print("Finding elements...")
    while True:
        container = bot.find_container()
        items = container.find_elements(By.XPATH, './li')
        for i in items:
            try:
                house = i.find_element(By.XPATH, './/a[2]')
                link = house.get_attribute('href')
                list_links.append(link)
                time.sleep(2)
                container_2= bot.driver.find_element(By.XPATH, '//div[@class="Slideshow-slides SlideshowCarousel"]')
                items_2 = container_2.find_elements(By.XPATH, './/img')
                img_links = []
                for i in items_2:
                    try:
                        link = i.get_attribute('src')
                        img_links.append(link)
                    except:
                        print('No src found')
            except:
                print('No href found, skipping this property')
        
        try:
            bot.button_click('//a[@class="btn paging-next"]')
        except NoSuchElementException:
            print("end of list")
            break

    #GRAB INFO FROM EACH LINK AND STORE

    prop_dict = {"fr-id": [],
            "id": [],
            "Link": [],
             "Summary": [],
            "Address": [],
            "Price": [],
            "Image links": []
            }
    for link in list_links:
        im = link[-6:]
        im2 = link[28:35]
        prop_dict["fr-id"].append(im2+im)
        id = uuid.uuid4()
        prop_dict["id"].append(id)
        bot.driver.get(link)
        prop_dict["Link"].append(link)
        time.sleep(1)
        summary = bot.driver.find_element(By.XPATH, '//div[@class="prop-heading-brief"]')
        prop_dict["Summary"].append(summary.text)
        time.sleep(1)
        info = bot.driver.find_element(By.XPATH, '//div[@class="prop-summary-row"]')
        address = info.find_element(By.XPATH, './/h1')
        prop_dict["Address"].append(address.text)
        time.sleep(1)
        price = bot.driver.find_element(By.XPATH, '//div[@class="prop-price"]')
        prop_dict["Price"].append(price.text)
        prop_dict["Image links"].append(img_links)

    df = pd.DataFrame(prop_dict)
    df

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/ryanhughes/.wdm/drivers/chromedriver/mac64/103.0.5060.53/chromedriver] found in cache
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_49062/3617797094.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = Chrome(ChromeDriverManager().install())
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_49062/3617797094.py:56: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = self.driver.find_element_by_xpath(xpath)
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_49062/3617797094.py:48: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  rent = self.driver.find_element_by_xpath('//*[@id="s

Finding elements...
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this property
No href found, skipping this p

In [81]:
df

,fr-id,id,Link,Summary,Address,Price,Image links
0,56-park766769,356b045d-bddf-48ea-a3a1-61499a3fed59,https://www.propertypal.com/56-park-avenue-bel...,3 Bed Terrace House To Rent,"56 PARK AVENUE,\nBELFAST BT4 1JJ",£795\n/ month + fees,[https://media.propertypal.com/sd/p/765569/323...
1,apt-1-2548636,7a3e5312-7359-4960-a5b0-b25eeef92aa2,https://www.propertypal.com/apt-1-223d-belmont...,1 Bed Apartment To Rent,"APT 1, 223D BELMONT ROAD,\nBELFAST BT4 2AH",£725\n/ month + fees,[https://media.propertypal.com/sd/p/765569/323...
2,1a-care765807,edcaedb2-b746-45b1-9289-500f81017b7b,https://www.propertypal.com/1a-carew-street-be...,2 Bed End-terrace House To Rent,"1A CAREW STREET,\nBelfast BT4 1HQ",£600\n/ month,[https://media.propertypal.com/sd/p/765569/323...
3,13-bath765788,cfe68122-c6b4-452e-8174-6180653e8971,https://www.propertypal.com/13-bathgate-drive-...,4 Bed Terrace House To Rent,"13 BATHGATE DRIVE,\nBelfast BT4 2BA",£895\n/ month + fees,[https://media.propertypal.com/sd/p/765569/323...
4,flat-3-532728,3af0229e-76eb-4052-8c6f-afd8db1a18cf,https://www.propertypal.com/flat-3-152-upper-n...,1 Bed Apartment To Rent,"FLAT 3, 152 UPPER NEWTOWNARDS ROAD,\n152 Upper...",£550\n/ month + fees,[https://media.propertypal.com/sd/p/765569/323...
5,229-par765499,dddf9a11-cdbe-4947-a79b-22e1d17dc171,https://www.propertypal.com/229-park-avenue-be...,4 Bed Mid-terrace House To Rent,"229 PARK AVENUE,\nBelfast BT4 1LN","£1,100\n/ month + fees",[https://media.propertypal.com/sd/p/765569/323...
6,9-pims-765427,0abefca7-004a-421c-8e55-668cd6cde3b4,https://www.propertypal.com/9-pims-avenue-belf...,3 Bed Terrace House To Rent,"9 PIMS AVENUE,\nBELFAST BT4 1PJ",£750\n/ month + fees,[https://media.propertypal.com/sd/p/765569/323...
7,apt-3-g765358,dfe82fcb-86e4-45df-a781-2b4fff434eb2,https://www.propertypal.com/apt-3-goldstone-ha...,2 Bed 1st Floor Apartment To Rent,"APT 3, GOLDSTONE HALL,\nBelfast BT4 1JE",£775\n/ month + fees,[https://media.propertypal.com/sd/p/765569/323...
8,53-uppe765329,03b35fc3-7746-4490-bf79-64d2122495c9,https://www.propertypal.com/53-upper-newtownar...,3 Bed Terrace House To Rent,"53 UPPER NEWTOWNARDS ROAD,\nBELFAST BT4 3HT",£850\n/ month + fees,[https://media.propertypal.com/sd/p/765569/323...
9,4-devon765273,41e6fb2b-191d-45f5-af10-edfdae968d9e,https://www.propertypal.com/4-devon-drive-syde...,2 Bed Terrace House To Rent,"4 DEVON DRIVE,\nSydenham, Belfast BT4 1LF",£575\n/ month,[https://media.propertypal.com/sd/p/765569/323...


In [82]:
prop_dict["Image links"]

[['https://media.propertypal.com/sd/p/765569/32335126.jpg',
  'https://media.propertypal.com/sd/p/765569/32335125.jpg',
  'https://media.propertypal.com/sd/p/765569/32335129.jpg',
  'https://media.propertypal.com/sd/p/765569/32335135.jpg'],
 ['https://media.propertypal.com/sd/p/765569/32335126.jpg',
  'https://media.propertypal.com/sd/p/765569/32335125.jpg',
  'https://media.propertypal.com/sd/p/765569/32335129.jpg',
  'https://media.propertypal.com/sd/p/765569/32335135.jpg'],
 ['https://media.propertypal.com/sd/p/765569/32335126.jpg',
  'https://media.propertypal.com/sd/p/765569/32335125.jpg',
  'https://media.propertypal.com/sd/p/765569/32335129.jpg',
  'https://media.propertypal.com/sd/p/765569/32335135.jpg'],
 ['https://media.propertypal.com/sd/p/765569/32335126.jpg',
  'https://media.propertypal.com/sd/p/765569/32335125.jpg',
  'https://media.propertypal.com/sd/p/765569/32335129.jpg',
  'https://media.propertypal.com/sd/p/765569/32335135.jpg'],
 ['https://media.propertypal.com/sd/

In [54]:
im = list_links[0][-6:]

im2 = list_links[0][28:35]

print(im+im2)

print(list_links[0])

76676956-park
https://www.propertypal.com/56-park-avenue-belfast/766769


In [2]:
import selenium
from selenium import webdriver
import pandas as pd
from selenium.webdriver import Chrome
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import time


class Scraper:
    def __init__(self, url: str = 'https://www.propertypal.com') -> None:
        self.driver = Chrome(ChromeDriverManager().install())
        self.driver.get(url)
        
    def ivan_accept_cookies(self, xpath:str = '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]'):
        
        try:
            time.sleep(1)
            WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))

            button = self.driver.find_element(By.XPATH, xpath)
            button.click()
        except TimeoutException:
            print("No Cookies Found")

    def accept_cookies(self, xpath:str = '//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]'):
        button = self.driver.find_element(By.XPATH, xpath)
        button.click()

    
    def search_word(self, xpath, text):

        search = self.driver.find_element(By.XPATH, xpath)
        self.driver.implicitly_wait(10)
        ActionChains(self.driver).move_to_element(search).click(search).perform()
        ActionChains(self.driver).send_keys(text).perform()

    def scroll_down(self):
        
        self.driver.execute_script("window.scrollTo(0, 540)") 
    
    def search_rent(self):
        rent = self.driver.find_element_by_xpath('//*[@id="searchForm"]/div/div[2]/button[2]')
        rent.click()

    def search_sale(self):
        rent = self.driver.find_element_by_xpath('//*[@id="searchForm"]/div/div[2]/button[1]')
        rent.click()

    def button_click(self,xpath):
        button = self.driver.find_element_by_xpath(xpath)
        button.click()
    
    def find_container(self, xpath: str = '//*[@id="body"]/div[3]/div/div[1]/div/ul'):
        self.container = self.driver.find_element(By.XPATH, xpath)
        return self.container

In [61]:
bot = Scraper()

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/103.0.5060.53/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/ryanhughes/.wdm/drivers/chromedriver/mac64/103.0.5060.53]
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_49062/100464251.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = Chrome(ChromeDriverManager().install())


In [76]:
container_2= bot.driver.find_element(By.XPATH, '//div[@class="Slideshow-slides SlideshowCarousel"]')
items = container_2.find_elements(By.XPATH, './/img')

[<selenium.webdriver.remote.webelement.WebElement (session="311458f89aee918ba583c6341a5e6c50", element="a9a330fe-0d75-4536-90a3-806426284c6f")>,
 <selenium.webdriver.remote.webelement.WebElement (session="311458f89aee918ba583c6341a5e6c50", element="f025e020-77bb-4171-8f52-a740d75a4429")>,
 <selenium.webdriver.remote.webelement.WebElement (session="311458f89aee918ba583c6341a5e6c50", element="cc7adcbf-1f3e-447c-b157-01622ab65921")>,
 <selenium.webdriver.remote.webelement.WebElement (session="311458f89aee918ba583c6341a5e6c50", element="4aa129fd-1ff7-4310-b277-e5a45a01c172")>]

In [66]:
items = bot.find_container('//div[@class="Slideshow-slides SlideshowCarousel"]')


In [77]:
container_2= bot.driver.find_element(By.XPATH, '//div[@class="Slideshow-slides SlideshowCarousel"]')
items = container_2.find_elements(By.XPATH, './/img')
img_links = []
for i in items:
    try:
        link = i.get_attribute('src')
        img_links.append(link)
    except:
        print('No src found')
    

In [78]:
bot.driver.get(img_links[0])

In [33]:
img_links

['https://media.propertypal.com/sd/p/764607/32259497.jpg',
 'https://media.propertypal.com/sd/p/764607/32259499.jpg',
 'https://media.propertypal.com/sd/p/764607/32259501.jpg',
 'https://media.propertypal.com/sd/p/764607/32259505.jpg']

In [57]:
import urllib.request

url = img_links[0]

r = urllib.request.urlopen(url)
with open("test_img.jpg", "wb") as f:
    f.write(r.read())

HTTPError: HTTP Error 403: Forbidden

In [59]:
import urllib.request
for url in img_links:

    req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    with open("image", "wb") as f:
        with urllib.request.urlopen(req) as r:
            f.write(r.read())

In [60]:

img_links

['https://media.propertypal.com/sd/p/764607/32259497.jpg',
 'https://media.propertypal.com/sd/p/764607/32259499.jpg',
 'https://media.propertypal.com/sd/p/764607/32259501.jpg',
 'https://media.propertypal.com/sd/p/764607/32259505.jpg']

In [43]:
list_UUID = []
for i in list_links:
    id = uuid.uuid4()
    list_UUID.append(id)
list_UUID

[UUID('2d80ad2f-786c-483d-92aa-fd1677316891'),
 UUID('0e02d07c-d303-4884-bd57-034708a8c6c4'),
 UUID('78a47e88-29f0-49af-bf41-1d227b4664e7'),
 UUID('fece02a6-f83a-49b8-b405-76a2da47dd64'),
 UUID('4ceb9dfd-6ca4-490a-bded-d8ea198fe463'),
 UUID('a296d491-8e7f-4f8a-9620-5df711cab5f3'),
 UUID('adfb8360-b539-437c-8daf-36bd135ffb6b'),
 UUID('0db7e6b0-c827-419c-9692-8b1e2d133dec'),
 UUID('7709d2bd-cf4f-4bf0-b410-7ce7e5d168fe'),
 UUID('f48a90b9-54f1-41da-a634-9ecaace25380'),
 UUID('398e2572-a765-4862-8bf2-0f1604a1116c'),
 UUID('4a838e38-f1e1-43a8-814e-bb588af1b32a'),
 UUID('4cd3f1ac-b3af-43f9-a119-de591268a2d9'),
 UUID('c3d31853-bcd7-44a7-8510-5239fe754420'),
 UUID('f52ae5b0-725a-48cb-b7e5-d013b38a54d3'),
 UUID('d9434206-a50c-48bb-95b9-1c1c116ff394'),
 UUID('ee488d6d-7e6e-47e6-be7c-72ba702b09d7'),
 UUID('d4e1eec9-7c53-49f3-8e63-48164c767b05'),
 UUID('5fa99b9d-a07c-4e3e-a6a5-0da11dae2a32'),
 UUID('6ad2dce6-a406-456a-9738-4238c070df51'),
 UUID('eafe051f-cc03-4810-88b5-ae215408cd38'),
 UUID('28b7b1